In [ ]:
import requests
import torch
from PIL import Image
from io import BytesIO
import os

os.chdir("/home/ubuntu/4catalyzer-hackathon")

from diffusers import StableDiffusionInstructPix2PixPipeline

device = "cuda"
# model_id_or_path = "runwayml/stable-diffusion-v1-5"
model_id_or_path = "./opt/run-overnight"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=False, ignore_mismatched_sizes=True)
pipe = pipe.to(device)


prompts = {
    "h2h": "This is an image of a normal brain MRI scan from axial view. It should be grey scale and clearly show brain structure. Do not modify the input image.",
    "h2uh": "Given an input image of a normal brain MRI scan from axial view, modify the image by inserting a brain lesion within the MRI image. This brain lesion is caused by an ischaemic stroke. The lesion should appear relatively lighter compared to its surroundings brain tissue in the image.",
    "uh2uhm": "Given an input image of an unhealthy brain MRI scan from axial view that contains brain lesions, modify the image to colour the lesions light red.",
    "h2uhm": "Given an input image of a normal brain MRI scan from axial view, modify the image by inserting a brain lesion within the MRI image, and colour the lesions light red. This brain lesion is caused by an ischaemic stroke.",
    "uhm2uhm": "This is an image of an unhealthy brain MRI scan from axial view that contains brain lesions. The lesions are coloured light red."
}

Keyword arguments {'ignore_mismatched_sizes': True} are not expected by StableDiffusionInstructPix2PixPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [52]:
# # Assuming the 'data' folder is in the same directory as your script
data_folder = "./data/train/"

# # Replace 'sketch-mountains-input.jpg' with the actual filename if needed
image_filename = 'healthy_images/sub-ON21834/caseON21834_dwi_slice_40.png'

# Replace 'sketch-mountains-input.jpg' with the actual filename if needed
# image_filename = 'lesion_images/sub-strokecase0002/case0002_dwi_slice_30.png'

# Build the full path to the image file
image_path = os.path.join(data_folder, image_filename)

# Open and resize the image
init_image = Image.open(image_path).convert("RGB")
# init_image = init_image.resize((768, 512))

# prompt = prompts["h2uhm"]
prompt = "modify the image by inserting one brain lesion within the MRI image, and highlight the brain lesion in bright red color."

generator = torch.Generator("cuda").manual_seed(40)


start_steps = 20
max_num_inference_steps = 200
os.system("rm -rf ./viz/")
os.system("mkdir ./viz/")
for steps in range(start_steps, max_num_inference_steps, 20):
    healthy_output_file = os.path.basename(image_filename)
    output_path = f"./viz/{healthy_output_file}"
    init_image.save(output_path)
    
    images = pipe(prompt, image=init_image,
                  num_inference_steps = steps,
                  image_guidance_scale = 5,
                  guidance_scale = 1,
                  num_images_per_prompt=1,
                  generator=generator).images
    for idx, image in enumerate(images):
        pred_output_path = f"./viz/{os.path.splitext(os.path.basename(image_filename))[0]}-generated-{idx}-step-{steps}.jpg"
        image.save(pred_output_path)

  0%|          | 0/20 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/40 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/60 [00:00<?, ?it/s]

...

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/180 [00:00<?, ?it/s]

## Visualize results

In [37]:
import os
import cv2

def create_video_from_directory(image_directory, output_video_path, frame_rate):
    image_files = sorted([f for f in os.listdir(image_directory) if f.endswith('.jpg')])  # Modify the extension as needed

    if not image_files:
        print("No image files found in the directory.")
        return

    first_image_path = os.path.join(image_directory, image_files[0])
    image = cv2.imread(first_image_path)
    height, width, _ = image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for the output video, modify as needed
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for image_file in image_files:
        image_path = os.path.join(image_directory, image_file)
        image = cv2.imread(image_path)
        out.write(image)

    out.release()
    print(f"Video created successfully at {output_video_path}")

# Example usage:
image_directory = './viz/'
output_video_path = './viz/video-3.mp4'
frame_rate = 1  # Adjust the frame rate as needed

create_video_from_directory(image_directory, output_video_path, frame_rate)


Video created successfully at ./viz/video-3.mp4
